In [5]:
from driftdb.connectors import GithubConnector
from dotenv import load_dotenv
from github import Github
import os

load_dotenv("../../.env")

gh_token = os.getenv("GH_TOKEN")
if gh_token is None:
    print("GitHub token not found! Create a .env file a the root with a GH_TOKEN variable.")
    exit(1)
github_client = Github(gh_token, timeout=60)
repo_name = os.getenv("REPO") or "gh_org/repo"
github_connector = GithubConnector(github_client=github_client, github_repository_name=repo_name, assignees=["Sammy"])


In [6]:

import pandas as pd


def formatDF(dict):
    df = pd.DataFrame(dict)
    df['unique_key'] = df.apply(lambda row: row['date'] + '-' + row['name'], axis=1)
    column_order = ['unique_key'] + [col for col in df.columns if col != 'unique_key']
    df = df.reindex(columns=column_order)
    return df


table_name = "path/to/metric_name_15.csv"


# Store metric for the first time
dataMonth1 = {"name": ["Alice", "Bob", "Charlie"], "date": ["2022-12","2023-01","2023-01"], "age": [25, 30, 35]}
github_connector.snapshot_table(
    table_dataframe= formatDF(dataMonth1),
    table_name= table_name,
)

driftdb.connectors.github_connector - INFO - Table found. Updating it
driftdb.connectors.github_connector - INFO - Change detected
driftdb.connectors.github_connector - INFO - Update: DRIFT
driftdb.connectors.github_connector - INFO - https://github.com/Samox/data-history/commit/634b5eb93649943806bde02b9e93904ac4208b85


In [7]:
# ## Introduce new data for 2023-02
dataMonth2 = {"name": ["Alice", "Bob", "Charlie", "Didier", "Philipe", "Antoine"], "date": ["2022-12","2023-01","2023-01","2023-02","2023-02","2023-02"], "age": [25, 30, 35, 40, 40, 40]}

github_connector.snapshot_table(
    table_dataframe= formatDF(dataMonth2),
    table_name= table_name,
)

driftdb.connectors.github_connector - INFO - Table found. Updating it
driftdb.connectors.github_connector - INFO - Change detected
driftdb.connectors.github_connector - INFO - Update: NEW DATA
driftdb.connectors.github_connector - INFO - https://github.com/Samox/data-history/commit/ea2b901d95d19ed87690b2c450d9e13f2ed59ce9


In [8]:
# ## Introduce new data for 2023-03 and a drift on 2020-02
dataMonth3 = {"name": ["Alice", "Bob", "Charlie", "Didier", "Philipe", "Antoine", "Clement", "Cyril", "Victor"], "date": ["2022-12","2023-01","2023-01","2023-02","2023-02","2023-02","2023-03","2023-03","2023-03"], "age": [25, 30, 36, 40, 42, 40, 45, 45, 46]}

github_connector.snapshot_table(
    table_dataframe= formatDF(dataMonth3),
    table_name= table_name,
)

driftdb.connectors.github_connector - INFO - Table found. Updating it
driftdb.connectors.github_connector - INFO - Change detected
driftdb.connectors.github_connector - INFO - Update: NEW DATA
driftdb.connectors.github_connector - INFO - https://github.com/Samox/data-history/commit/fe378db98efc3b498853254b6e838c45518e55d4
driftdb.connectors.github_connector - INFO - Update: DRIFT
driftdb.connectors.github_connector - INFO - https://github.com/Samox/data-history/commit/db23b8ac7548eca81e76d55400b8b582b7e16bf1


In [9]:
# ## No new data. Adds a drift for Philipe, and remove all other drifts
from driftdb.drift_evaluator.drift_evaluators import AlertDriftEvaluator


dataMonth3And1Day = {"name": ["Alice", "Bob", "Charlie", "Didier", "Philipe", "Antoine", "Clement", "Cyril", "Victor"], "date": ["2022-12","2023-01","2023-01","2023-02","2023-02","2023-02","2023-03","2023-03","2023-03"], "age": [25, 30, 35, 40, 42, 40, 45, 45, 46]}
github_connector.snapshot_table(
    table_dataframe= formatDF(dataMonth3And1Day),
    table_name= table_name,
    drift_evaluator=AlertDriftEvaluator()
)

driftdb.connectors.github_connector - INFO - Table found. Updating it
driftdb.connectors.github_connector - INFO - Change detected
driftdb.connectors.github_connector - INFO - Update: DRIFT
driftdb.connectors.github_connector - INFO - Branch drift/2023-11-20-15-28-56/path-to-metric-name-15-csv doesn't exist. Creating it...
driftdb.connectors.github_connector - INFO - Checkout branch: drift/2023-11-20-15-28-56/path-to-metric-name-15-csv from branch: main
driftdb.connectors.github_connector - INFO - https://github.com/Samox/data-history/commit/d2259f7d29d0599df63db4807288652b3716f283
driftdb.connectors.github_connector - INFO - Pull request created: https://github.com/Samox/data-history/pull/205
driftdb.connectors.github_connector - INFO - Assignee Sammy does not exist


In [10]:
# ## No new data. Adds a drift for Philipe, and remove all other drifts
dataMonth3And2Day = {"name": ["Alice", "Alixe", "Bob", "Charlie", "Didier", "Philipe", "Antoine", "Clement", "Cyril", "Victor"], "date": ["2022-12","2022-12","2023-01","2023-01","2023-02","2023-02","2023-02","2023-03","2023-03","2023-03"], "age": [25, 25, 30, 35, 40, 42, 40, 45, 45, 46]}

github_connector.snapshot_table(
    table_dataframe= formatDF(dataMonth3And2Day),
    table_name= table_name,
    drift_evaluator=AlertDriftEvaluator()
)


driftdb.connectors.github_connector - INFO - Table found. Updating it
driftdb.connectors.github_connector - INFO - Change detected
driftdb.connectors.github_connector - INFO - Update: DRIFT
driftdb.connectors.github_connector - INFO - Branch drift/2023-11-20-15-29-06/path-to-metric-name-15-csv doesn't exist. Creating it...
driftdb.connectors.github_connector - INFO - Checkout branch: drift/2023-11-20-15-29-06/path-to-metric-name-15-csv from branch: main
driftdb.connectors.github_connector - INFO - https://github.com/Samox/data-history/commit/2bb8258b5ab0bd927cecaaddcc6afd707c8cd0ef
driftdb.connectors.github_connector - INFO - Pull request created: https://github.com/Samox/data-history/pull/206
driftdb.connectors.github_connector - INFO - Assignee Sammy does not exist
